## Getting started with NetCov

NetCov is tool that can be used with [Batfish](https://github.com/batfish/batfish) to analyze test coverage for network configurations. Given a set of Batfish queries, it analyzes which lines of configurations have/have not been covered. The result can be used to:
- Evaluate the rigorousness of network testing
- Help identify the blind spots of network testing

NetCov is written in Python and can be used in concert with [pybatfish](https://pybatfish.readthedocs.io/en/latest/notebooks/interacting.html), Batfish's Python client API.

#### To install NetCov:
`
pip install netcov
`
A virtual environment and Python version 3.7 is recommended.

NetCov leverages LCOV to produce HTML-format coverage report. LCOV can be installed via `brew install lcov` (MacOS) or `sudo apt install lcov` (Ubantu).

#### Step 1: Import
NetCov encapsulates coverage tracking into a set of hooked pybatfish APIs. To start with, import as the following.
For an existing pybatfish script, replace `from pybatfish.client.session import Session` with the following:

In [1]:
from netcov import NetCovSession as Session

#### Step 2: Interact with Batfish service as normal

To proceed, please ensure that your Batfish service is running on localhost.

In [2]:
bf = Session(host="localhost")
bf.init_snapshot("fattree4", overwrite=True);

#### Step 3: Network testing using Batfish queries as normal
NetCov automatically tracks coverage for supported types of queries, such as `bf.q.routes()` and `bf.q.traceroute()`.

In [3]:
def test_default_route_presence():
    """Check that all routers have the default route."""
    fattree_nodes = bf.q.nodeProperties(nodes="/edge|aggr|core/").answer().frame()['Node'].to_list()
    for node in fattree_nodes:
        r = bf.q.routes(nodes=node, network="0.0.0.0/0").answer().frame()
        assert len(r.index) > 0
        
test_default_route_presence()

In [4]:
from pybatfish.datamodel import HeaderConstraints
def test_pingmesh():
    """Check that all pairs of leaf routers can reach each other."""
    leaf_nodes = bf.q.nodeProperties(nodes="/edge/").answer().frame()['Node'].to_list()
    for src in leaf_nodes:
        for dst in leaf_nodes:
            tr = bf.q.traceroute(
                startLocation=src, 
                headers=HeaderConstraints(
                    srcIps=f"{src}[Loopback0]",
                    dstIps=f"{dst}[Loopback0]"), 
                maxTraces=1
            ).answer().frame()
            assert tr.Traces[0][0][-1].node == dst
            assert tr.Traces[0][0][-1][-1].action == 'ACCEPTED'
            assert tr.Traces[0][0][-1][-1].detail.interface == 'Loopback0'    

test_pingmesh()

#### Step 4: Coverage results

To print coverage metrics into the console:

In [5]:
bf.cov.result()

Configuration coverage:
    Covered lines:                         640/784 (81.63%)
Breakdown:
    bgp neighbor:                          136/136 (100.00%)
    interface:                             492/636 (77.36%)
    route-map-clause:                      8/8 (100.00%)
    route-map:                             8/8 (100.00%)
    ipv4 prefix-list:                      4/4 (100.00%)


To view line-level coverage result as annotations on source configurations, use the command below and check out the generated HTML report under `SNAPSHOT_PATH/coverage` directory:

In [6]:
bf.cov.html_report()